# Pull And Run Docker Image
## Using Triton and Bria-attribution-model

In [ ]:

import os
import requests

api_token = '<API_TOKEN>'

def get_repository_access_password():
    """Fetch ECR credentials from localhost endpoint"""
    try:
        headers = {"api_token":  api_token}
        url ="https://engine.prod.bria-api.com"
        response = requests.get(f"{url}/v2/image/attribution/embedding_repository_access", headers=headers)
        response.raise_for_status()
        result = response.json()["result"]
        password = result["password"]
        
        print(f"Successfully fetched ECR credentials, expires at: {result['expiration']}")
        return password
    except Exception as e:
        print(f"Failed to fetch ECR credentials: {e}")
        raise

password = get_repository_access_password()
! echo "$password" | docker login --username AWS --password-stdin 300465780738.dkr.ecr.us-east-1.amazonaws.com

! docker pull 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest
! docker kill $(docker ps -q --filter expose=8002)
! docker run -d   --runtime=nvidia   --shm-size=16gb   -p 8000:8000   -p 8001:8001   -p 8002:8002 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest

Successfully fetched ECR credentials, expires at: 2025-08-29T00:55:49.897000+00:00
Removing login credentials for 300465780738.dkr.ecr.us-east-1.amazonaws.com
WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded
Error response from daemon: pull access denied for 300465780738.dkr.ecr.us-east-1.amazonaws.com/add_asset, repository does not exist or may require 'docker login': denied: User: arn:aws:sts::300465780738:assumed-role/embeddingsAccessRole/docker-puller is not authorized to perform: ecr:BatchGetImage on resource: arn:aws:ecr:us-east-1:300465780738:repository/add_asset because no identity-based policy allows the ecr:BatchGetImage action
"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers


# Client Side

## Installations

In [ ]:
import os
import uuid
import requests
import tritonclient.http as httpclient
from PIL import Image
from embedder import BRIAEmbedder

## Inference Request

In [ ]:
url = '<BRIA_EMBEDDINGS_URL>:8000'
image_path = '<IMAGE_PATH>'

triton_client = httpclient.InferenceServerClient(
    url=url,
    # ssl=True,
    # ssl_context_factory=gevent.ssl._create_default_https_context,
)
image = Image.open(image_path)
bria_pipeline = BRIAEmbedder(triton_client)
img_embeddings = bria_pipeline.run_on_image(image)


## Send Attribution API

In [ ]:
model_version = os.environ.get("model_version", "2.3")
attribution_endpoint = os.environ.get("attribution_endpoint", "httsp://engine.prod.bria-api.com/v2/image/attribution/by_embedding")
api_token = '<API_TOKEN>'

try:
    print(f"Read data from queue")
    embeddings_uid = str(uuid.uuid4())
    print(f"embeddings_uid: {embeddings_uid}")
    request_body = {
        "embeddings_base64": img_embeddings,
        "embeddings_uid": embeddings_uid,
        "model_version": 2.3,
        "agent":  api_token,
        "model_name": 'replace-backgroud',
    }
    
    headers = {
        "api_token":  api_token
    }

    if attribution_endpoint:
        response = requests.post(attribution_endpoint, json=request_body, headers=headers)
        if response.status_code != 202 or response.status_code != 200:
            print(f"API request failed with status code {response.status_code}: {response.text}")
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
        else:
            print(f"Request body: {request_body}")
except Exception as ex:
    print(f"Error processing record: {ex}")
    raise